# Analysing Smartphone Prices

In [14]:
from statistics import stdev, median

import requests
from lxml import html
from urllib import parse
import re

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

## 1. Loading Devices Names

In [2]:
RELATIVE_PATH_TO_DATA_FOLDER = './data/'
filename = 'devices_using_ribonapp_data'
filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + filename + '.csv'
devices_df = pd.read_csv(filename_with_path, skiprows=1, names=['model_name', 'old_price_range'])
devices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129 entries, 0 to 1128
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   model_name       1129 non-null   object
 1   old_price_range  1129 non-null   object
dtypes: object(2)
memory usage: 17.8+ KB


### Cleaning Duplicates and Missing Values

In [3]:
devices_df.drop_duplicates(['model_name'], inplace=True)
devices_df.dropna(axis=0, inplace=True)
devices_df

,model_name,old_price_range
0,2 A2005,Low
1,3,Low
2,3T,Low
3,4047D,Low
4,5,Low
...,...,...
1124,nova 4,Low
1125,t1_8p0lte,Low
1126,vivo 1724,Low
1127,vivo X9,Low


### Filtering Non Informative Names

In [4]:
informative_names_mask = devices_df['model_name'].str.len() > 1
devices_df = devices_df[informative_names_mask]
devices_df

,model_name,old_price_range
0,2 A2005,Low
2,3T,Low
3,4047D,Low
5,5051J,Low
6,5056D,Low
...,...,...
1124,nova 4,Low
1125,t1_8p0lte,Low
1126,vivo 1724,Low
1127,vivo X9,Low


# Searching for Prices

In [ ]:


url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()
url_list = [url_base + parse.quote(item_name) for item_name in item_list]

# We can use the website filtering to help our search
# Let's filter for only devices in the tag 'Novo'
novo_filter_xpath = '/html/body/main/div[2]/div/aside/section[2]/dl[@id="id_ITEM_CONDITION"]/dd[1]/a/@href'
products_articles_criteria = '@class="results-item highlighted article stack product "'
product_information_criteria = '@class="item__info-container highlighted "'
item_prices_criteria = '@class="price__container"'
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'
headers = {'User-Agent': user_agent}

found_elements = []
for i, url in tqdm(enumerate(url_list)):
    webpage_requested = requests.get(url, headers=headers, allow_redirects=True)
    if not webpage_requested or not webpage_requested.content:
        found_elements.append([item_list[i], url, [], [], []])
        continue

    webpage_tree = html.fromstring(webpage_requested.content)


    improved_url = webpage_tree.xpath(novo_filter_xpath)
    if improved_url:
        improved_url = improved_url[0]
        webpage_requested = requests.get(improved_url)
        webpage_tree = html.fromstring(webpage_requested.content)

    
    webproducts_names_xpath = '/html/body/main/div[2]/div/section/ol/li[' + products_articles_criteria + ']/div/div[' + product_information_criteria + ']/div/h2/a/span/text()'
    webproducts_prices_xpath = '/html/body/main/div[2]/div/section/ol/li[' + products_articles_criteria +\
        ']/div/div['+ product_information_criteria + ']/div/div[' + item_prices_criteria + ']/div/span[2]/text()'

    webproducts_names_found = webpage_tree.xpath(webproducts_names_xpath)
    webproducts_prices_found = webpage_tree.xpath(webproducts_prices_xpath)
    if len(webproducts_names_found) > len(webproducts_prices_found):
        webproducts_names_found = webproducts_names_found[:len(webproducts_prices_found)]
    elif len(webproducts_names_found) < len(webproducts_prices_found = ):
        webproducts_prices_found = webproducts_prices_found[:len(webproducts_names_found)]
    
    found_elements.append([item_list[i], url, improved_url, webproducts_names_found, webproducts_prices_found])
print(len(found_elements))

In [ ]:
print(found_elements)

In [ ]:
# Explode/Split column into multiple rows
new_df = pd.DataFrame(df.City.str.split('|').tolist(), index=df).stack()
new_df = new_df.reset_index([0, 'EmployeeId'])
new_df.columns = ['EmployeeId', 'City']

In [ ]:
url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()[:5]

# TODO ADD OPTION TO SEARCH ONLY NEW PRODUCTS 
# title="Novo" href="(.*?)">
url_list = [url_base + parse.quote(item_name) for item_name in item_list]
found_search_contents = []

In [ ]:
url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()

# TODO ADD OPTION TO SEARCH ONLY NEW PRODUCTS 
# title="Novo" href="(.*?)">
url_list = [url_base + parse.quote(item_name) for item_name in item_list]
found_search_contents = []
item_count = 1
for url, item_original_name in zip(url_list, item_list):
    print(item_count)
    
    item_count += 1
    r = requests.get(url)
    text_content = str(r.text)
    # We can use the website filtering to help our search
    # Let's filter for only devices in the tag 'Novo'
    improved_url = re.search(
             r"""id="id_ITEM_CONDITION" role="listbox">[\s\S]+?title=['"]Novo['"] href=['"](\S+)['"]\s*>""", text_content)
    if improved_url:
        improved_url = improved_url.groups()[0]
        r = requests.get(improved_url)
        text_content = str(r.text)
     
    # Let's filter the are to search for only search results

    result_field_text_content = re.search(r'<ol id="searchResults"(.+?)<\/ol>', text_content)
    if result_field_text_content:
        result_field_text_content = result_field_text_content.groups()[0]
    else:
        result_field_text_content = None
    found_search_contents.append([item_original_name, url, result_field_text_content])

extracted_data_df = pd.DataFrame(data=found_search_contents, columns=['item_original_name','url', 'page_search_content'])
extracted_data_df

## Storing Raw Price Information

In [5]:
extracted_filename = 'scrapy_found_devices'
extracted_filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + extracted_filename + '.csv'

In [ ]:
extracted_data_df.to_csv(extracted_filename_with_path, index=False)

## Treating Data

In [6]:
device_raw_extracted_data_df = pd.read_csv(extracted_filename_with_path, header=0)
device_raw_extracted_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22272 entries, 0 to 22271
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_original_name  22272 non-null  object
 1   url                 22272 non-null  object
 2   improved_url        22272 non-null  object
 3   device_name         22272 non-null  object
 4   device_price        22272 non-null  object
dtypes: object(5)
memory usage: 870.1+ KB


In [15]:
%time
device_raw_extracted_data_df.dropna(axis=0, inplace=True)
device_raw_extracted_data_df['device_price'] = pd.to_numeric(device_raw_extracted_data_df['device_price'], errors='coerce').fillna(0).astype(np.int64)
device_raw_extracted_data_df

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 12.2 µs


,item_original_name,url,improved_url,device_name,device_price
0,5051j,https://celulares.mercadolivre.com.br/5051j,[],PLACA 2CHIP DO CELULAR ALCATEL MODELO 5051J. ...,75
1,5051j,https://celulares.mercadolivre.com.br/5051j,[],SMARTPHONE ALCATEL POP 4 PREMIUM (5051J). TEL...,140
2,5051j,https://celulares.mercadolivre.com.br/5051j,[],CELULAR ALCATEL POP,200
3,6039j,https://celulares.mercadolivre.com.br/6039j,[],"ALCATEL ONE TOUCH IDOL 3 6039J - 13MP, 4G, DU...",649
4,6039j,https://celulares.mercadolivre.com.br/6039j,[],"ALCATEL ONE TOUCH IDOL 3 6039J 13MP, 4G, DUAL...",599
...,...,...,...,...,...
22267,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],SAMSUNG GALAXY S9 DUAL SIM 128 GB PRETO-MEIA-...,2999
22268,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],SONY XPERIA XZ PREMIUM DUAL SIM 64 GB PRETO-M...,3740
22269,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],XIAOMI REDMI NOTE 8 DUAL SIM 32 GB BRANCO-LUA...,1499
22270,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],MOTOROLA MOTO G7 DUAL SIM 64 GB CERAMIC BLACK...,1399


In [16]:
INVALID_WORDS = set(['DEFEITO', 'TROCO', 'QUEBRADO', 'CABO', 'CONECTOR', 'PEÇA'])
valid_rows_mask = ~device_raw_extracted_data_df['device_name'].str.contains('|'.join(INVALID_WORDS))
device_extracted_data_df = device_raw_extracted_data_df[valid_rows_mask]
device_extracted_data_df

,item_original_name,url,improved_url,device_name,device_price
0,5051j,https://celulares.mercadolivre.com.br/5051j,[],PLACA 2CHIP DO CELULAR ALCATEL MODELO 5051J. ...,75
1,5051j,https://celulares.mercadolivre.com.br/5051j,[],SMARTPHONE ALCATEL POP 4 PREMIUM (5051J). TEL...,140
2,5051j,https://celulares.mercadolivre.com.br/5051j,[],CELULAR ALCATEL POP,200
3,6039j,https://celulares.mercadolivre.com.br/6039j,[],"ALCATEL ONE TOUCH IDOL 3 6039J - 13MP, 4G, DU...",649
4,6039j,https://celulares.mercadolivre.com.br/6039j,[],"ALCATEL ONE TOUCH IDOL 3 6039J 13MP, 4G, DUAL...",599
...,...,...,...,...,...
22267,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],SAMSUNG GALAXY S9 DUAL SIM 128 GB PRETO-MEIA-...,2999
22268,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],SONY XPERIA XZ PREMIUM DUAL SIM 64 GB PRETO-M...,3740
22269,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],XIAOMI REDMI NOTE 8 DUAL SIM 32 GB BRANCO-LUA...,1499
22270,nova-4,https://celulares.mercadolivre.com.br/nova-4,[],MOTOROLA MOTO G7 DUAL SIM 64 GB CERAMIC BLACK...,1399


In [18]:
extracted_data_df = device_extracted_data_df.groupby("item_original_name").median()
extracted_data_df

,device_price
item_original_name,
3t,2400.0
5051j,140.0
5t,2549.0
6039j,624.0
6t,2350.0
...,...
zenfone3,650.0
zenfone4-%28ze554kl%29,850.0
zs621kl%29-zenfone%25205z%2520%2528zs620kl,748.0


##  Storing Results

In [ ]:
extracted_filename = 'extracted_device_data_2'
extracted_filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + extracted_filename + '.csv'

In [ ]:
extracted_data_df.to_csv(extracted_filename_with_path, index=False)

#  Analysing Prices

In [ ]:
device_extracted_data_df = pd.read_csv(extracted_filename_with_path, header=0)
device_extracted_data_df.info()

## Prices Distribution

In [ ]:
device_extracted_data_df.sort_values(by='item_price', inplace=True)
device_extracted_data_df.plot.scatter(x=0, y='item_price', figsize=(12, 12))

## Prices Binning

In [ ]:
device_extracted_data_df.describe()
device_extracted_data_df.plot.hist(by=['item_price'], bins=5, figsize=(8, 8))


# Price  Ranges
![Brazilian IBGE Salaries Ranges Research](./assets/ibge_brazilian_salary_ranges.jpg)

In [ ]:
ibge_price_ranges_labels = ['very_low', 'low', 'medium', 'high', 'very_high']
salaries_ibge_ranges = [(0, 1908), (1909, 2862), (2863, 5724), (5725, 3566)]
ibge_price_ranges = [0]
for i, salarie_range in enumerate(salaries_ibge_ranges):
    threshold_price = (salarie_range[0] + salarie_range[1])//3
    # As the values are so low, we have to increment them with the previous threshold
    threshold_price += ibge_price_ranges[i-1]
    ibge_price_ranges.append(threshold_price)
ibge_price_ranges.append(float('inf'))

device_extracted_data_df['ibge_price_range'] = pd.cut(device_extracted_data_df['item_price'], bins=ibge_price_ranges, labels=ibge_price_ranges_labels)
groups = device_extracted_data_df.groupby('ibge_price_range', sort=False)
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group['item_original_name'], group['item_price'], marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()

In [ ]:
device_extracted_data_df

## Comparing with old price ranges

In [ ]:
device_extracted_data_df = device_extracted_data_df.merge(right=devices_df, left_on='item_original_name', right_on='model_name', how='inner').drop(columns='model_name')
groups = device_extracted_data_df.groupby('old_price_range', sort=False)
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group['item_original_name'], group['item_price'], marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()